In [ ]:
##This is Anfield

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

def clean_text(text):
    text = text.replace("\u2018", "'").replace("\u2019", "'")
    text = text.replace("\u201C", '"').replace("\u201D", '"')
    text = text.replace("\u2013", "-").replace("\u2014", "-") 
    text = unicodedata.normalize("NFKD", text) 
    return text.strip()

url = "https://www.thisisanfield.com/liverpool-fc-news/"
driver.get(url)

wait = WebDriverWait(driver, 30)
try:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2.entry-title a")))
except:
    print("❌ Timeout: Articles not found. Trying alternative methods...")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

articles = soup.select("article")
news_data = []

for article in articles:
    headline_element = article.select_one("h2.entry-title a")
    date_element = article.select_one("time.entry-date, time.published, span.posted-on time")
    
    if headline_element and date_element:
        headline = clean_text(headline_element.text) 
        news_url = headline_element["href"]
        date_published = clean_text(date_element.text)
        
        time_published = date_element.get("datetime", "Unknown")
        if time_published != "Unknown":
            time_published = datetime.fromisoformat(time_published.replace("Z", "+00:00")).strftime("%H:%M:%S")
        
        print(f"Extracted: {headline} | Date: {date_published} | Time: {time_published}")
        
        news_data.append([date_published, time_published, headline, news_url])

for i, (_, _, _, news_url) in enumerate(news_data):
    driver.get(news_url)
    time.sleep(2) 
    
    article_html = driver.page_source
    article_soup = BeautifulSoup(article_html, "html.parser")
    
    body_elements = article_soup.select("div.elementor-widget-theme-post-content p, div.elementor-widget-theme-post-content h2, div.elementor-widget-theme-post-content blockquote")
    body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])
    
    news_data[i].append(body_content)

driver.quit()
dfa = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "URL", "Body Context"])
dfa.to_csv("thisisanfield_news.csv", index=False, encoding="utf-8-sig")
print("✅ Scraping completed successfully! Data saved as 'thisisanfield_news.csv'")

Extracted: Jeremie Frimpong transfer links explained after meeting with Liverpool trio | Date: Mar 18, 2025 | Time: 15:48:26
Extracted: Julian Alvarez to Liverpool transfer update - Atletico Madrid respond on 'official offer' | Date: Mar 18, 2025 | Time: 13:36:56
Extracted: Liverpool's youngest-ever player now part of YouTube league aged 28 | Date: Mar 18, 2025 | Time: 12:47:40
Extracted: Conor Bradley injury update in mixed news for Liverpool - "He's frustrated" | Date: Mar 18, 2025 | Time: 11:19:53
Extracted: Robbie Fowler says "unbelievable statement" sums up Arne Slot at Liverpool | Date: Mar 18, 2025 | Time: 10:46:09
Extracted: Federico Chiesa given advantage no other Liverpool player has this month | Date: Mar 18, 2025 | Time: 10:07:31
Extracted: Luis Diaz 'transfer interest' from Saudi Arabia - "concrete possibility" to leave | Date: Mar 18, 2025 | Time: 09:45:04
Extracted: Liverpool FC international break: 20 players called up for March fixtures | Date: Mar 18, 2025 | Time: 09:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

def clean_text(text):
    return text.strip()

csv_file = "thisisanfield_news_backup_link.csv"
df = pd.read_csv(csv_file)

if "Body Context" not in df.columns:
    df["Body Context"] = "No Content"

for index, row in df.iterrows():
    url = row["URL"]
    print(f"🔍 Scraping: {url}")

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code != 200:
            print(f"⚠️ Error {response.status_code}: Skipping {url}")
            df.at[index, "Body Context"] = f"Error {response.status_code}"
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        body_elements = soup.select("div.elementor-widget-theme-post-content p, div.entry-content p")
        body_content = "\n".join([clean_text(element.text) for element in body_elements if element.text.strip()])

        if not body_content:
            print(f"⚠️ Warning: No body content found for {url}")
            df.at[index, "Body Context"] = "No Content"
        else:
            df.at[index, "Body Context"] = body_content[:500] 

        print(f"✅ Successfully Scraped Row {index + 1}:")
        print(f"🔎 First 10 words: {' '.join(body_content.split()[:10])}...\n")
        time.sleep(random.uniform(2, 5))

    except Exception as e:
        print(f"❌ Error: {e}")
        df.at[index, "Body Context"] = "Scraping Error"

    if (index + 1) % 10 == 0:
        df.to_csv("thisisanfield_news_temp.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Temporary save at row {index + 1}")


df.to_csv("thisisanfield_news_updated.csv", index=False, encoding="utf-8-sig")
print("🎉 Scraping Completed! Data saved in 'thisisanfield_news_updated.csv'")

🔍 Scraping: https://www.thisisanfield.com/2025/03/jeremie-frimpong-transfer-links-explained-after-meeting-with-liverpool-trio/
✅ Successfully Scraped Row 1:
🔎 First 10 words: Bayer Leverkusen right-back Jeremie Frimpong has been linked with a...

🔍 Scraping: https://www.thisisanfield.com/2025/03/julian-alvarez-to-liverpool-transfer-update-atletico-madrid-respond-on-official-offer/
✅ Successfully Scraped Row 2:
🔎 First 10 words: Julian Alvarez was subject of interest from Liverpool last summer,...

🔍 Scraping: https://www.thisisanfield.com/2025/03/liverpools-youngest-ever-player-now-part-of-youtube-league-aged-28/
✅ Successfully Scraped Row 3:
🔎 First 10 words: The record for Liverpool’s youngest-ever player has not been broken...

🔍 Scraping: https://www.thisisanfield.com/2025/03/conor-bradley-injury-update-in-mixed-news-for-liverpool-hes-frustrated/
✅ Successfully Scraped Row 4:
🔎 First 10 words: Conor Bradley is not expected to make an imminent return...

🔍 Scraping: https://www.this